# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.sample(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
415,415,namibe,-15.1961,12.1522,72.25,76,2,9.95,AO,1676140686
61,61,bambous virieux,-20.3428,57.7575,79.21,73,40,8.05,MU,1676140543
48,48,faanui,-16.4833,-151.7500,81.48,75,19,14.14,PF,1676140538
516,516,hun,29.1268,15.9477,54.14,35,100,6.26,LY,1676140732
166,166,guasdualito,7.2424,-70.7324,91.36,49,40,2.30,VE,1676140585
132,132,upernavik,72.7868,-56.1549,-15.14,92,100,4.99,GL,1676140570
197,197,camacha,33.0833,-16.3333,60.22,67,20,6.91,PT,1676140433
183,183,hithadhoo,-0.6000,73.0833,80.96,69,98,12.24,MV,1676140592
135,135,victoria,22.2855,114.1577,68.38,93,98,11.01,HK,1676140489
114,114,lata,40.1629,-8.3327,49.51,56,100,10.89,PT,1676140561


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng","Lat", geo = True, tiles = "OSM", frame_width = 800,frame_height = 400,
    size = 'Humidity', scale = 0.8, color = 'm')

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values

good_weather = city_data_df.loc[city_data_df['Max Temp'] < 75,:]

good_weather = good_weather.loc[good_weather['Max Temp'] > 50,:]
good_weather = good_weather.loc[good_weather['Humidity'] < 75,:]
good_weather = good_weather.loc[good_weather['Cloudiness'] < 85,:]
good_weather = good_weather.loc[good_weather['Cloudiness'] > 30,:]

# Drop any rows with null values
good_weather.dropna(inplace=True)

# Display sample data

good_weather


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,punta arenas,-53.1500,-70.9167,55.51,47,75,14.97,CL,1676140483
9,9,ushuaia,-54.8000,-68.3000,55.06,50,75,9.22,AR,1676140272
24,24,ribeira grande,38.5167,-28.7000,58.77,62,75,14.97,PT,1676140525
87,87,port lincoln,-34.7333,135.8667,61.70,71,68,12.26,AU,1676140552
176,176,marsa matruh,31.3525,27.2453,57.40,58,40,11.50,EG,1676140590
310,310,praia,14.9215,-23.5087,73.94,60,75,17.27,CV,1676140645
370,370,lompoc,34.6391,-120.4579,57.69,71,75,15.99,US,1676140403
457,457,umm kaddadah,13.6017,26.6876,74.79,16,82,13.40,SD,1676140708
514,514,marrakesh,31.6315,-8.0083,66.27,39,40,8.05,MA,1676140505
521,521,baghdad,33.3406,44.4009,53.51,47,40,5.75,IQ,1676140519


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = good_weather[['City', 'Lat', 'Lng', 'Country', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df['Hotel Name'] = ''
hotel_df.reset_index(inplace=True, drop=True)

# Display sample data

hotel_df.sample(10)

,City,Lat,Lng,Country,Humidity,Hotel Name
4,marsa matruh,31.3525,27.2453,EG,58,
2,ribeira grande,38.5167,-28.7000,PT,62,
9,baghdad,33.3406,44.4009,IQ,47,
1,ushuaia,-54.8000,-68.3000,AR,50,
3,port lincoln,-34.7333,135.8667,AU,71,
6,lompoc,34.6391,-120.4579,US,71,
8,marrakesh,31.6315,-8.0083,MA,39,
7,umm kaddadah,13.6017,26.6876,SD,16,
5,praia,14.9215,-23.5087,CV,60,
0,punta arenas,-53.1500,-70.9167,CL,47,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
radius = 10000
params = {}
params['apiKey'] = geoapify_key
params['categories'] = 'accommodation.hotel'


# Print a message to follow up the hotel search

print("Starting hotel search")

# Iterate through the hotel_df DataFrame


for index, row in hotel_df.iterrows():
    
    
    # get latitude, longitude from the DataFrame
    
    latitude = hotel_df.loc[index, 'Lat']
    longitude = hotel_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    
    
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
   
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
   
    print(f"{hotel_df.loc[index, 'City']} - near hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
punta arenas - near hotel: Hotel Lacolet
ushuaia - near hotel: Apart Hotel Aires del Beagle
ribeira grande - near hotel: Hospedaria JSF - 3 Estrelas
port lincoln - near hotel: Boston Hotel
marsa matruh - near hotel: صقور
praia - near hotel: Casa Sodadi
lompoc - near hotel: Embassy Suites by Hilton Lompoc Central Coast
umm kaddadah - near hotel: No hotel found
marrakesh - near hotel: No hotel found
baghdad - near hotel: AL Bedaa Hotel


,City,Lat,Lng,Country,Humidity,Hotel Name
0,punta arenas,-53.1500,-70.9167,CL,47,Hotel Lacolet
1,ushuaia,-54.8000,-68.3000,AR,50,Apart Hotel Aires del Beagle
2,ribeira grande,38.5167,-28.7000,PT,62,Hospedaria JSF - 3 Estrelas
3,port lincoln,-34.7333,135.8667,AU,71,Boston Hotel
4,marsa matruh,31.3525,27.2453,EG,58,صقور
5,praia,14.9215,-23.5087,CV,60,Casa Sodadi
6,lompoc,34.6391,-120.4579,US,71,Embassy Suites by Hilton Lompoc Central Coast
7,umm kaddadah,13.6017,26.6876,SD,16,No hotel found
8,marrakesh,31.6315,-8.0083,MA,39,No hotel found
9,baghdad,33.3406,44.4009,IQ,47,AL Bedaa Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
hotels_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 400,
    size = 'Humidity',
    scale = 0.8,
    color = 'm',
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
hotels_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)